# DineSafe Data Analysis

## Data Wrangling

In [273]:
#relevant libraries
import os
import re
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import xml.etree.ElementTree as ET
import json
import numpy as np
#xmltodict is very useful here, let's get it

In [ ]:
pip install xmltodict

In [649]:
from xmltodict import parse
from collections import ChainMap

#need to flatten the XML as pandas has a hard time parsing this with from_XML()
raw_dict = parse(open('ds_od_xml 2022-11-13.xml', 'rb'))
raw_dict1 = dict(ChainMap(*raw_dict.values()))
raw_data_1 = pd.json_normalize(raw_dict1,record_path='ESTABLISHMENT')

#need to re-encode this file because this file is encoded in latin-1 and Pandas can't decode it for some reason
parser = ET.XMLParser(encoding='latin-1')
raw_data_2 = ET.parse('ds 2022-01-13.xml',parser)
raw_data_2.write('out.xml', encoding='utf-8')
raw_dict = parse(open('out.xml', 'rb'))
raw_dict2 = dict(ChainMap(*raw_dict.values())) #remove the initial dinesafe header key
raw_data_2 = pd.json_normalize(raw_dict2,record_path='ESTABLISHMENT')

#this older file is in UTF-8, hurray, but still too highly nested for Pandas
raw_dict = parse(open('dinesafe 2019-04-08.xml', 'rb'))
raw_dict3 = dict(ChainMap(*raw_dict.values()))
raw_data_3 = pd.json_normalize(raw_dict3,record_path='ROW')

#finally, hand-coded chains
chains = pd.read_csv('dinesafe 2019-04-08 chains.csv', encoding='latin-1', dtype={'Establishment ID':'int','Inspection ID':'int','Company/Franchise':'str','Name':'str','Type':'str','Address':'str','Lat':'float','Long':'float','Status':'str','Minimum inspections per year':'int','Infraction details':'str','Inspection date':'str','Severity':'str','Action':'str','Court outcome':'str','Amount fined':'float'})

In [640]:
#processing the first file

#need to flatten individual inspection results, this notebook uses outdated pandas so can't just use orient parameter

#first remove null values - ones with no inspections
no_nulls = raw_data_1['INSPECTION'].dropna()

#now flatten it
flattened_inspections = pd.DataFrame.from_dict(no_nulls.explode().to_dict(),orient='index')

#now we have individual infractions per inspection, need to flatten those...
#infraction values of NaN in this one are straight passes, let's put them in a separate series
no_infractions = flattened_inspections.loc[flattened_inspections['INFRACTION'].isnull(),:]

#now split out single-infraction inspections
has_inspections = flattened_inspections.dropna()['INFRACTION'].to_dict()
one_infraction_keys = []
multi_infraction_keys = []
for key in has_inspections:
    if isinstance(has_inspections[key],list):
        multi_infraction_keys.append(key)
    else: one_infraction_keys.append(key)

#now flatten single-infraction inspections
one_infraction_dict = has_inspections.copy()
for row in multi_infraction_keys:
    one_infraction_dict.pop(row)
one_infraction = pd.DataFrame.from_dict(one_infraction_dict,orient='index')

#great, now we have single infractions per inspection split out, let's tackle multiple infractions per inspection
#isolate and split multi-infractions
multi_infraction_dict = has_inspections.copy()
for key in one_infraction_keys:
    multi_infraction_dict.pop(key)
multi_infraction = pd.DataFrame.from_dict(multi_infraction_dict,orient='index').stack().reset_index()
#now convert to dataframe
multi_infraction_rows = pd.DataFrame.from_dict(multi_infraction[0].to_dict(),orient='index')
#and stick it back to we can reference the index of inspections
multi_infraction_combined = pd.concat([multi_infraction,multi_infraction_rows],axis='columns')

#now the infraction tables are as follows:
#one_infraction.head() #severity deficiency action conviction outcome fines
#no_infractions.head() #status date infraction - don't need to concat this, it's already in there
#multi_infraction_combined.head() #raw data key, severity, deficiency, action etc

#need to get all the infractions back into a single inspection dataframe then back to the restaurant list
infractions_1 = pd.concat([flattened_inspections,one_infraction],axis='columns')
infractions_1 = infractions_1.join(multi_infraction_combined.reindex(multi_infraction_combined['level_0']),on=infractions_1.index,how='left',lsuffix='multi')
infractions_1['SEVERITY'].fillna(infractions_1['SEVERITYmulti'],inplace=True)
infractions_1['DEFICIENCY'].fillna(infractions_1['DEFICIENCYmulti'],inplace=True)
infractions_1['ACTION'].fillna(infractions_1['ACTIONmulti'],inplace=True)
infractions_1['CONVICTION_DATE'].fillna('None',inplace=True) #all are completely null here so not losing anything
infractions_1['COURT_OUTCOME'].fillna('None',inplace=True) #all are completely null here so not losing anything
infractions_1['AMOUNT_FINED'].fillna('None',inplace=True) #all are completely null here so not losing anything
infractions_1.drop(['key_0','INFRACTION','SEVERITYmulti','DEFICIENCYmulti','ACTIONmulti','CONVICTION_DATEmulti','COURT_OUTCOMEmulti','AMOUNT_FINEDmulti','level_1',0],axis='columns',inplace=True)

#now we have a list of infractions, join it back to the list of restaurants
cleaned_data_1 = raw_data_1.join(infractions_1,how='outer',rsuffix='.inf')

#clean up
cleaned_data_1['STATUS'].fillna(cleaned_data_1['STATUS.inf'],inplace=True)
cleaned_data_1['DATE'].fillna(cleaned_data_1['INSPECTION.DATE'],inplace=True)
cleaned_data_1['SEVERITY'].fillna(cleaned_data_1['INSPECTION.INFRACTION.SEVERITY'],inplace=True)
cleaned_data_1['DEFICIENCY'].fillna(cleaned_data_1['INSPECTION.INFRACTION.DEFICIENCY'],inplace=True)
cleaned_data_1['ACTION'].fillna(cleaned_data_1['INSPECTION.INFRACTION.ACTION'],inplace=True)
cleaned_data_1['CONVICTION_DATE'].fillna('None',inplace=True) #all are completely null in the infraction set here so not losing anything
cleaned_data_1['COURT_OUTCOME'].fillna('None',inplace=True) #all are completely null in the infraction set here so not losing anything
cleaned_data_1['AMOUNT_FINED'].fillna('None',inplace=True) #all are completely null in the infraction set here so not losing anything
cleaned_data_1.drop(['level_0','INSPECTION','INSPECTION.STATUS','INSPECTION.DATE','INSPECTION.INFRACTION','INSPECTION.INFRACTION.SEVERITY','INSPECTION.INFRACTION.DEFICIENCY','INSPECTION.INFRACTION.ACTION','INSPECTION.INFRACTION.CONVICTION_DATE','INSPECTION.INFRACTION.COURT_OUTCOME','INSPECTION.INFRACTION.AMOUNT_FINED','STATUS.inf','level_0'],axis='columns',inplace=True)

#check
cleaned_data_1.head()

,ID,NAME,TYPE,ADDRESS,LATITUDE,LONGITUDE,STATUS,DATE,SEVERITY,DEFICIENCY,ACTION,CONVICTION_DATE,COURT_OUTCOME,AMOUNT_FINED
0,10752656,# HASHTAG INDIA RESTAURANT,Food Take Out,1871 O'CONNOR DR,43.72199,-79.30349,Pass,2022-08-18,NaN,NaN,NaN,None,None,None
1,9008018,'K' STORE,Food Store (Convenience/Variety),99 CARLTON ST,43.66205,-79.37747,Pass,NaN,NaN,NaN,NaN,None,None,None
2,10510325,0109 Dessert + Chocolate,Restaurant,"2190 MCNICOLL AVE, -109",43.81477,-79.29491,Pass,2022-11-07,NaN,NaN,NaN,None,None,None
3,10737088,1 HOTELS - CASA MADERA,Restaurant,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2022-10-06,M - Minor,FAIL TO ENSURE EQUIPMENT SURFACE SANITIZED AS ...,Notice to Comply,None,None,None
3,10737088,1 HOTELS - CASA MADERA,Restaurant,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2022-10-06,M - Minor,FAIL TO ENSURE EQUIPMENT SURFACE SANITIZED AS ...,Notice to Comply,None,None,None


In [641]:
#processing the second file
#need to flatten individual inspection results, this notebook uses outdated pandas so can't just use orient parameter

#first remove null values - ones with no inspections
no_nulls = raw_data_2['INSPECTION'].dropna()

#now flatten it
flattened_inspections = pd.DataFrame.from_dict(no_nulls.explode().to_dict(),orient='index')

#now we have individual infractions per inspection, need to flatten those...
#infraction values of NaN in this one are straight passes, let's put them in a separate series
no_infractions = flattened_inspections.loc[flattened_inspections['INFRACTION'].isnull(),:]

#now split out single-infraction inspections
has_inspections = flattened_inspections.dropna()['INFRACTION'].to_dict()
one_infraction_keys = []
multi_infraction_keys = []
for key in has_inspections:
    if isinstance(has_inspections[key],list):
        multi_infraction_keys.append(key)
    else: one_infraction_keys.append(key)

#now flatten single-infraction inspections
one_infraction_dict = has_inspections.copy()
for row in multi_infraction_keys:
    one_infraction_dict.pop(row)
one_infraction = pd.DataFrame.from_dict(one_infraction_dict,orient='index')

#great, now we have single infractions per inspection split out, let's tackle multiple infractions per inspection
#isolate and split multi-infractions
multi_infraction_dict = has_inspections.copy()
for key in one_infraction_keys:
    multi_infraction_dict.pop(key)
multi_infraction = pd.DataFrame.from_dict(multi_infraction_dict,orient='index').stack().reset_index()
#now convert to dataframe
multi_infraction_rows = pd.DataFrame.from_dict(multi_infraction[0].to_dict(),orient='index')
#and stick it back to we can reference the index of inspections
multi_infraction_combined = pd.concat([multi_infraction,multi_infraction_rows],axis='columns')

#now the infraction tables are as follows:
#one_infraction.head() #severity deficiency action conviction outcome fines
#no_infractions.head() #status date infraction - don't need to concat this, it's already in there
#multi_infraction_combined.head() #raw data key, severity, deficiency, action etc

#need to get all the infractions back into a single inspection dataframe then back to the restaurant list
infractions_2 = pd.concat([flattened_inspections,one_infraction],axis='columns')
infractions_2 = infractions_2.join(multi_infraction_combined.reindex(multi_infraction_combined['level_0']),on=infractions_2.index,how='left',lsuffix='multi')
infractions_2['SEVERITY'].fillna(infractions_2['SEVERITYmulti'],inplace=True)
infractions_2['DEFICIENCY'].fillna(infractions_2['DEFICIENCYmulti'],inplace=True)
infractions_2['ACTION'].fillna(infractions_2['ACTIONmulti'],inplace=True)
infractions_2['CONVICTION_DATE'].fillna('None',inplace=True) #all are completely null here so not losing anything
infractions_2['COURT_OUTCOME'].fillna('None',inplace=True) #all are completely null here so not losing anything
infractions_2['AMOUNT_FINED'].fillna('None',inplace=True) #all are completely null here so not losing anything
infractions_2.drop(['key_0','INFRACTION','SEVERITYmulti','DEFICIENCYmulti','ACTIONmulti','CONVICTION_DATEmulti','COURT_OUTCOMEmulti','AMOUNT_FINEDmulti','level_1',0],axis='columns',inplace=True)

#now we have a list of infractions, join it back to the list of restaurants
cleaned_data_2 = raw_data_2.join(infractions_2,how='outer',rsuffix='.inf')

#clean up
cleaned_data_2['STATUS'].fillna(cleaned_data_2['STATUS.inf'],inplace=True)
cleaned_data_2['DATE'].fillna(cleaned_data_2['INSPECTION.DATE'],inplace=True)
cleaned_data_2['SEVERITY'].fillna(cleaned_data_2['INSPECTION.INFRACTION.SEVERITY'],inplace=True)
cleaned_data_2['DEFICIENCY'].fillna(cleaned_data_2['INSPECTION.INFRACTION.DEFICIENCY'],inplace=True)
cleaned_data_2['ACTION'].fillna(cleaned_data_2['INSPECTION.INFRACTION.ACTION'],inplace=True)
cleaned_data_2['CONVICTION_DATE'].fillna('None',inplace=True) #all are completely null in the infraction set here so not losing anything
cleaned_data_2['COURT_OUTCOME'].fillna('None',inplace=True) #all are completely null in the infraction set here so not losing anything
cleaned_data_2['AMOUNT_FINED'].fillna('None',inplace=True) #all are completely null in the infraction set here so not losing anything
cleaned_data_2.drop(['level_0','INSPECTION','INSPECTION.STATUS','INSPECTION.DATE','INSPECTION.INFRACTION','INSPECTION.INFRACTION.SEVERITY','INSPECTION.INFRACTION.DEFICIENCY','INSPECTION.INFRACTION.ACTION','INSPECTION.INFRACTION.CONVICTION_DATE','INSPECTION.INFRACTION.COURT_OUTCOME','INSPECTION.INFRACTION.AMOUNT_FINED','STATUS.inf','level_0'],axis='columns',inplace=True)

#check
cleaned_data_2.head()

,ID,NAME,TYPE,ADDRESS,LATITUDE,LONGITUDE,STATUS,DATE,SEVERITY,DEFICIENCY,ACTION,CONVICTION_DATE,COURT_OUTCOME,AMOUNT_FINED
0,9008018,'K' STORE,Food Store (Convenience/Variety),99 CARLTON ST,43.66205,-79.37747,Pass,NaN,NaN,NaN,NaN,None,None,None
1,10510325,0109 Dessert + Chocolate,Restaurant,"2190 MCNICOLL AVE, -109",43.81477,-79.29491,Pass,NaN,NaN,NaN,NaN,None,None,None
2,10737088,1 HOTELS - CASA MADERA,Restaurant,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2021-08-19,NaN,NaN,NaN,None,None,None
3,10737083,1 HOTELS - FLORA LOUNGE (LOBBY BAR),Cocktail Bar / Beverage Room,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2021-08-19,NaN,NaN,NaN,None,None,None
4,10737091,1 HOTELS - HARRIET'S,Restaurant,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2021-08-19,NaN,NaN,NaN,None,None,None


In [650]:
#processing the third (oldest) file which is in a different format
#first normalize column names
cleaned_data_3 = raw_data_3.rename(columns={'ESTABLISHMENT_ID':'ID','ESTABLISHMENT_NAME':'NAME','ESTABLISHMENTTYPE':'TYPE','ESTABLISHMENT_ADDRESS':'ADDRESS','ESTABLISHMENT_STATUS':'STATUS','INSPECTION_DATE':'DATE','INFRACTION_DETAILS':'DEFICIENCY'},inplace=False)
#and that's it! what a much easier thing to work with

In [653]:
cleaned_data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88819 entries, 0 to 88818
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ROW_ID                       88819 non-null  object
 1   ID                           88819 non-null  object
 2   INSPECTION_ID                88819 non-null  object
 3   NAME                         88819 non-null  object
 4   TYPE                         88819 non-null  object
 5   ADDRESS                      88819 non-null  object
 6   LATITUDE                     88819 non-null  object
 7   LONGITUDE                    88819 non-null  object
 8   STATUS                       88819 non-null  object
 9   MINIMUM_INSPECTIONS_PERYEAR  88819 non-null  object
 10  DEFICIENCY                   60262 non-null  object
 11  DATE                         88819 non-null  object
 12  SEVERITY                     60262 non-null  object
 13  ACTION                       60

In [707]:
#great, all the data is fully flattened, let's combine them together
inspections=pd.concat([cleaned_data_1,cleaned_data_2,cleaned_data_3])

#now join chain assignments on establishment ID/ID
#reindex chains to make establishment ID the index for join
chains_condensed = pd.concat([chains['Company/Franchise'],chains['Establishment ID']],axis='columns').set_index('Establishment ID').loc[~chains.index.duplicated(), :]
inspection_chain = inspections.join(chains_condensed,on=inspections['ID'].astype('int'))

#reset the index as there are duplicates per inspection row, and remove overlapping inspections
cleaned_dataset = inspection_chain.reset_index().rename({'index':'inspection_index'},axis='columns').loc[~inspection_chain.index.duplicated(), :]

#datetime assignment to dates
cleaned_dataset['DATE']=pd.to_datetime(cleaned_dataset['DATE'],errors='ignore',yearfirst=True)

cleaned_dataset.head()
#potential future work: new stores have opened since, so for the top 100 chains, try to find this keyword in each store name with this dict:

,inspection_index,ID,NAME,TYPE,ADDRESS,LATITUDE,LONGITUDE,STATUS,DATE,SEVERITY,DEFICIENCY,ACTION,CONVICTION_DATE,COURT_OUTCOME,AMOUNT_FINED,ROW_ID,INSPECTION_ID,MINIMUM_INSPECTIONS_PERYEAR,Company/Franchise
0,0,10752656,# HASHTAG INDIA RESTAURANT,Food Take Out,1871 O'CONNOR DR,43.72199,-79.30349,Pass,2022-08-18,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
1,1,9008018,'K' STORE,Food Store (Convenience/Variety),99 CARLTON ST,43.66205,-79.37747,Pass,NaT,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,'K' STORE
2,2,10510325,0109 Dessert + Chocolate,Restaurant,"2190 MCNICOLL AVE, -109",43.81477,-79.29491,Pass,2022-11-07,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
3,3,10737088,1 HOTELS - CASA MADERA,Restaurant,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2022-10-06,M - Minor,FAIL TO ENSURE EQUIPMENT SURFACE SANITIZED AS ...,Notice to Comply,None,None,None,NaN,NaN,NaN,NaN
5,4,10737083,1 HOTELS - FLORA LOUNGE (LOBBY BAR),Cocktail Bar / Beverage Room,550 WELLINGTON ST W,43.64284,-79.40167,Pass,2022-10-06,S - Significant,OPERATE FOOD PREMISE - MECHANICAL EQUIPMENT NO...,Corrected During Inspection,None,None,None,NaN,NaN,NaN,NaN


In [708]:
#export
cleaned_dataset.to_csv('cleaned_dataset.csv')